# **Pincode-Specific local restaurant Geo-tagging using Foursquare API**

## This project utilizes Foursquare API to answer the Questions like : How many types of foods are available in a restaurant? What type of restaurants are present in a particular area? Which is the most nearest to me with good rating?

In [ ]:
# Enter your Info
Country= "india"
State= 'karnataka'
City= 'bangalore'
Area= 'Agram'

In [ ]:
# 1
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# insall lxml to read an html file
!pip install lxml

Solving environment: done

# All requested packages already installed.

  Using cached https://files.pythonhosted.org/packages/dd/ba/a0e6866057fc0bbd17192925c1d63a3b85cf522965de9bc02364d08e5b84/lxml-4.5.0-cp36-cp36m-manylinux1_x86_64.whl
Target directory /home/spark/shared/user-libs/python3.6/lxml-4.5.0.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3.6/lxml already exists. Specify --upgrade to force replacement.


In [ ]:
# Read the html file
df = pd.read_html(r'https://www.mapsofindia.com/pincode/india/{}/{}/'.format(State, City), header = 1)
df = pd.DataFrame(df[0])
df =df.sort_values(by='Location', ascending=True)
# df= df.drop_duplicates(subset="Pincode", keep="last")

# Read address for each locality
Stat= df.iloc[0,2]
list=[]
for loc, pin, dis in zip(df['Location'], df['Pincode'], df['District']):
    try:
# Use nested try/ Except to handle the error[] alternatively and finally pass to final 'Except'
        try:
            address = '{}, {}'.format(pin, Stat)
            geolocator = Nominatim(user_agent="Ind_Explorer", timeout = 10)
            location = geolocator.geocode(address)
            lat = location.latitude
            lng = location.longitude
            list.append([(
                loc,
                pin,
                dis,
                lat,
                lng)])
    
        except:
            address = '{}, {}'.format(loc, Stat)
            geolocator = Nominatim(user_agent="BLR_explorer", timeout = 10)
            location = geolocator.geocode(address)
            lat = location.latitude
            lng = location.longitude
            list.append([(
                loc,
                pin,
                dis,
                lat,
                lng)])
    except:
        pass

fil_df = pd.DataFrame([item for venue in list for item in venue])
fil_df.columns = ['Location', 
                    'Pincode',
                    'District', 
                    'Latitude',
                    'Longitude']

# read client information
CLIENT_ID = 'T3MED4EITMTO2OCQXNLAPZPTXP3T0LIOONZ4FI2LYIG2ZU0H' # your Foursquare ID
CLIENT_SECRET = 'N2GKGR134RH1GAFSG4WPSB1EFRCJYOYCWED5CPGZZC3KDWRM' # your Foursquare Secret
VERSION = '20180604'

# Read the venues from Foursquare API
venues_list=[]
radius=1500
LIMIT =100

for loc, dis, lat, lng in zip(fil_df['Location'], fil_df['District'], fil_df['Latitude'], fil_df['Longitude']):
    
        # create the API request URL
    try:
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            loc, 
            dis,  
            x['venue']['name'], 
            x['venue']['categories'][0]['name'],
            x['venue']['location']['distance']) for x in results])
    except:
        pass
    
venue_df = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
venue_df.columns = ['Location', 
                    'District', 
                    'Landmark', 
                    'Category',
                    'Distance']
venue_df =venue_df.sort_values(by='Location', ascending=True)
venue_df.reset_index(drop=True, inplace = True)

# filter the food Dataframe
food_df= venue_df[venue_df['Category'].str.contains('Restaurant|Milk|Sweet|lunch|Kabab|Egg|Smoke|Pizza|Joint|Dim sum|Coffee|Smoke|Sweet shop|Café|Bistro|Hotel|Bar|Ice cream|Sandwich|Steak|BBQ|Liquor|Brewery|Joint|Burger|Candy|Tea|Cake|Bakery|Burrito|Snack|Bistro|Food|Breakfast|Juice|Dessert|Snack|Beer|Wine|Donut|Fast food|Salad|Diner|Fried|Chicken|Drink|Hookah|Chocolate', regex=True, case=False)]
food_df.reset_index(drop=True, inplace = True)
food_df.head()

# drop column, 'Category'
food_df.drop(['Category'], axis=1, inplace=True)

# segregate all unique locations
un_loc=food_df['Location'].unique()

# sort the subdataframes by distance, for each location
final_df=pd.DataFrame(columns=['Location', 'District', 'Landmark', 'Distance'])
for loc,lc in zip(food_df['Location'], un_loc):
    sbs = food_df[food_df['Location'] == lc]
    sbs =sbs.sort_values(by='Distance', ascending=True)
    final_df= final_df.append(sbs)
    
# drop column, 'Distance'
final_df.reset_index(drop=True, inplace = True)
final_df.drop(['Distance'], axis=1, inplace=True)

# Merge the venues for similar locations into a dictionary
lnd= final_df.groupby('Location')['Landmark'].agg(' ★ '.join)

# Drop duplicate location rows
final_df= final_df.drop_duplicates(subset="Location", keep="first")
final_df.reset_index(drop=True, inplace = True)

# insert collection of venues to the specified location in Dataframe
for i,j in zip(final_df['Landmark'], lnd):
    final_df.replace(i,j, inplace=True)
    
# drop column, 'District'
final_df.drop(['District'], axis=1, inplace=True)

# Export dataframe to csv file 
name= 'Fake_Zomato'
export_csv = final_df.to_csv ('{}.csv'.format(name), index = None, header=True)

# use get_close_matches to find area of interest and print final Info
import difflib
from difflib import get_close_matches

lib = difflib.get_close_matches(Area, final_df['Location'])

for i,j in zip(final_df['Location'], final_df['Landmark']):
    if i==lib[0]:
        print('\n\t\t\t\t\t❤❤{}❤❤\n\n{}\n'.format(i,j))

/opt/ibm/conda/miniconda3.6/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)



					❤❤Agram❤❤

Domino's Pizza ★ Khan Saheb ★ Infinitea ★ Ooty Chocolates ★ Tandoori Taal ★ The Kitchen Of Joy ★ California Burrito ★ Retro Lounge Bar ★ Corner House ★ Vietnamese Kitchen ★ Namdhari's Fresh ★ Kim Lee ★ Annapoorani ★ Mamagoto ★ Phobidden Fruit ★ Imperial Restaurant ★ Lavonne ★ Mother Cluckers ★ Chianti Ristorante & Wine Bar ★ Krispy Kreme Doughnuts ★ High Note Bar and Dining ★ Bricklane Grill ★ Vapour - Pub and Brewery ★ Chaipatty Teafe ★ Smoor Chocolates by Bliss ★ 1131 Bar N Kitchen ★ Bhartiya Jalpan ★ Bob's Bar ★ Bite Me Cupcakes ★ Loft38 Lounge ★ Cream Centre ★ Barbeque Nation (Lido Mall) ★ Hyatt Centric Mg Road Bangalore ★ flying spaghetti monster ★ Nagarjuna ★ Bangalore Mandarin ★ Little Italy ★ Starbucks ★ Domino's Pizza ★ Bliss Chocolate Lounge ★ Sri Udupi Park ★ Drops Total Spirits ★ The Ants Cafe ★ Harry's Singapore ★ Rajdhani ★ Hotel Shalimar Bar and Restaurant ★ The Tao Terraces ★ Esplanade Calcutta Cuisine ★ Creme & Crust ★ Meghana Foods ★ Yauatcha ★ Häagen